In [ ]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = ""

In [69]:
os.environ["USER_AGENT"] = "langchain-web-loader/1.0 (local dev)"


In [ ]:
os.environ["OLLAMA_API_KEY"]=""

In [ ]:
import os
from langchain.chat_models import init_chat_model
import ollama


# model = init_chat_model("gemma3:1b")
model = init_chat_model(
    "qwen2.5:3b", 
    model_provider="ollama"
)

In [ ]:
# from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
# embeddings = OllamaEmbeddings(model="gemma3:1b")
embeddings = OllamaEmbeddings(model="embeddinggemma")


# embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [73]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [74]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
# This acts as a filter to extract only the specific elements 
# It will extract only those elements where the class attribute is post-title, post-header and post-content
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load() # converting the content into Lang Chain document
# What happens internally

# 1. Sends an HTTP GET request to the URL

# 2. Downloads the HTML

# 3. Parses it with BeautifulSoup

# 4. Applies the SoupStrainer

# 5. Extracts text

# 6. Wraps it into LangChain Document objects

assert len(docs) == 1
# This is a sanity check:

# You expect exactly one document

# If the loader failed or split unexpectedly → assertion error

print(f"Total characters: {len(docs[0].page_content)}")

Total characters: 43047


In [75]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


In [76]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 63 sub-documents.


In [77]:
document_ids = vector_store.add_documents(documents=all_splits)


In [78]:
print(document_ids[:3])


['69ae3500-b4eb-440d-8b49-2654b68acee7', 'a00e1e72-1640-40a8-8843-0648b3b66069', 'eee37883-2ec7-4e4d-93d2-eb584e488c35']


In [79]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [80]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [81]:
from langchain.agents import create_agent


tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

In [84]:
query = (
    "What is the standard method for Task Decomposition?\n\n"
    "Once you get the answer, look up common extensions of that method."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the standard method for Task Decomposition?

Once you get the answer, look up common extensions of that method.
================================== Ai Message ==================================

I need to find out the standard method for Task Decomposition first. Let's retrieve information about it.
Tool Calls:
  retrieve_context (b8bc2747-4820-46e4-98ef-e50fad2c66cc)
 Call ID: b8bc2747-4820-46e4-98ef-e50fad2c66cc
  Args:
    query: standard method for Task Decomposition
  retrieve_context (824da8a5-da57-46c0-a1b0-5a3bfda19d1a)
 Call ID: 824da8a5-da57-46c0-a1b0-5a3bfda19d1a
  Args:
    query: common extensions of standard method for Task Decomposition
  retrieve_context (09713335-f32a-44a8-b337-8e4a21732036)
 Call ID: 09713335-f32a-44a8-b337-8e4a21732036
  Args:
    query: common extensions of standard method for Task Decomposition
================================= Tool Message ====================